<a href="https://colab.research.google.com/github/deemeetree/dfa/blob/master/Understanding_DFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Detrended Fluctuation Analysis** is used to compare fluctuations of a signal across different time scales. The log of the (sum of the?) fluctuations are plotted on the Y axis against the log of the time scale length (X axis). If the relation is linear, there's a power law relation between the signal and the time. That is, there are a few signals with significantly high values (e.g. amplitude) and then there's many signals with a low value — hence, the power law. 

How is it calculated? First, we import the necessary libraries:


In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

Then define numpy array. Let's say in the first case it consists of 16 values, each oscillates around the mean of 8.



In [ ]:
x = np.array([8, 10, 6, 9, 7, 5, 5, 11, 11, 8, 6, 7, 9, 10, 7, 9])

Let's plot this array to see the signal:

In [ ]:
plt.plot(x)
plt.show()

We see that this signal oscillates around the mean of 8 with the standard deviation of 1.9 and variance 3.6:

In [ ]:
# mean
print(x.mean())

# standard deviation
print(x.std())

# variance
variance = 0
for v in x:
  variance = variance + abs(v - x.mean())**2
print(variance)
print(variance / x.size)

# standard deviation from variance
stdev = math.sqrt(variance / (x.size))
print(stdev)


To calculate this signal's DFA we will take the following steps:

In [ ]:
# create an array with a cumulative sum of each deviation across the scale

y = np.cumsum(x - np.mean(x))
print(x)
print(y)

# this is basically transforming the normal process into random walk

plt.plot(y)
plt.show()


In [ ]:
# create the scales we're going to use
# i.e. how many numbers we're going to have in each scale

scales = (2**np.arange(2, 4.5, 0.5)).astype(np.int)
print(scales)

# prepare an array for fluctuations calculation for every scale

fluct = np.zeros(len(scales))
print(fluct)

Now we will add our cumulative fluctuation value for each window and calculate RMS (root mean square) for each:

In [ ]:
# making an array with data divided into time windows

# for each scale 

for e, sc in enumerate(scales):
  
  # how many times we have to divide the scale to fit the numbers in each
  # number of windows, number of elements in each window (rows, columns)
  shape = (y.shape[0]//sc, sc)
  print('shape', shape)
  
  # create an array with data divided in windows for each scale (rows, columns)
  X = np.lib.stride_tricks.as_strided(y,shape=shape)
  print('strides',X)

  # create a vector for each row (number columns)
  scale_ax = np.arange(sc)
  print('scale',scale_ax)

  # rows - the number of windows for each scale
  rms = np.zeros(X.shape[0])
  
    
  # for each value in the window (each column in a row)
  for w, xcut in enumerate(X):
    # calculate polynomial fit of the cumsum to x in 1D
    # e.g. make a straight line that fits values found in that window row
    coeff = np.polyfit(scale_ax, xcut, 1)
    print('window values', xcut)
    print('coeff', coeff)

    # use polyfit coefficients to build a straight line
    # this is how we detrend the data
    xfit = np.polyval(coeff, scale_ax)
    print('xfit', xfit)

    # root mean square of the difference between the value and the fit
    rms[w] = np.sqrt(np.mean((xcut-xfit)**2))
    # print('rms',e,rms[e])
    # which is the same as stdev
    # stdv = (xcut-xfit).std()
  
  print(rms)
  fluct[e] = np.sqrt(np.mean(rms**2))

# now we have data on all fluctuations
print('fluct',fluct)

alpha = np.polyfit(scales, fluct, 1)
alpha2 = np.polyfit(np.log2(scales), np.log2(fluct), 1)

fluctfit = 2**np.polyval(alpha2,np.log2(scales))


plt.loglog(scales,fluctfit, "r", basex=2,basey=2)
plt.loglog(scales,fluct, "bo",basex=2,basey=2)
plt.show()

fluctfit2 = np.polyval(alpha,scales)

plt.plot(scales,fluctfit2, "r")
plt.plot(scales,fluct, "bo")

plt.show()

print('alpha', alpha)
print('alpha2', alpha2)

# interpretation

# alpha = 0.5 - when the scale increases 2 times the sq root of variance increases about 1.5 times
# alpha = 1 - scale increases 2 times and variance grows 2 times - on the limit between tending to an average and exploding exponentially
# fractal means we find the same kind of logic we find in the small scale on the big scale
# alpha > 1.1 - complex means we increase the period of observation and the logic breaks — there is no stationery mean — things are changing too much (influence from the outside?)
